# Notebook 10: Full Pipeline — MAS + Post-Training + Evaluation

## Learning Objectives
- Build a complete end-to-end MAS pipeline
- Run on math benchmarks and collect traces
- Apply credit assignment
- Simulate post-training with collected signals
- Evaluate improvement

This notebook **ties together all previous notebooks** and mirrors the Argonne internship project.

## End-to-End Workflow

```
Step 1: Build MAS (Solver + Critic + Reviser + Verifier)
           ↓
Step 2: Run on GSM8K benchmark (20 problems)
           ↓
Step 3: Collect traces + outcome rewards
           ↓
Step 4: Credit assignment (Shapley + error localization)
           ↓
Step 5: Generate per-agent training signals
           ↓
Step 6: Simulate post-training (DPO with agent-specific data)
           ↓
Step 7: Evaluate improvement on held-out problems
```

In [ ]:
# !pip install torch transformers tqdm matplotlib

In [ ]:
import sys
sys.path.insert(0, '..')
import json, torch
import matplotlib.pyplot as plt
print('All imports ready!')

## Step 1: Build the MAS

In [ ]:
from src.agents import SolverAgent, CriticAgent, ReviserAgent, VerifierAgent
from src.orchestration.pipeline import PipelineOrchestrator
from src.orchestration.logger import TraceLogger

solver   = SolverAgent(agent_id='solver_0')
critic   = CriticAgent(agent_id='critic_0')
reviser  = ReviserAgent(agent_id='reviser_0')
verifier = VerifierAgent(agent_id='verifier_0')

pipeline = PipelineOrchestrator([solver, critic, reviser, verifier], max_rounds=2)
print('MAS pipeline ready with agents:', [a.agent_id for a in pipeline.agents])

## Step 2: Run on GSM8K Benchmark

In [ ]:
from src.data.gsm8k_loader import GSM8KLoader
from src.evaluation.metrics import MetricsTracker

loader = GSM8KLoader()
problems = loader.get_batch(0, 10)  # First 10 problems

tracker = MetricsTracker()
all_results = []

for i, prob in enumerate(problems):
    for a in [solver, critic, reviser, verifier]: a.reset()
    result = pipeline.run(prob['question'], ground_truth=float(prob['answer']))
    tracker.record(correct=bool(result['correct']), rounds=result['rounds'])
    all_results.append(result)
    status = 'CORRECT' if result['correct'] else 'WRONG'
    print(f'[{i+1:2d}] {status} | rounds={result["rounds"]} | ans={result["final_answer"]} (gt={prob["answer"]})')

print('\nMetrics:', tracker.summary())

## Step 3: Credit Assignment on Collected Traces

In [ ]:
from src.credit_assignment.shapley import ShapleyCalculator
from src.credit_assignment.error_localization import ErrorLocalizer

# Collect credit signals across all runs
agent_signals = {'solver_0': [], 'critic_0': [], 'reviser_0': [], 'verifier_0': []}

for result, prob in zip(all_results, problems):
    gt = float(prob['answer'])
    trace = pipeline.logger.messages if hasattr(pipeline.logger, 'messages') else []
    localizer = ErrorLocalizer(ground_truth=gt)
    # Simple heuristic: correct=+1 credit to all, wrong=-0.5 to solver
    correct = bool(result['correct'])
    for a_id in agent_signals:
        agent_signals[a_id].append(1.0 if correct else (-0.5 if a_id == 'solver_0' else 0.0))

import numpy as np
print('Mean credit signals per agent:')
for a_id, signals in agent_signals.items():
    print(f'  {a_id:15s}: {np.mean(signals):.3f} (n={len(signals)})')

## Step 4: Visualize Credit Distribution

In [ ]:
from src.evaluation.visualization import plot_agent_contributions

mean_signals = {a: float(sum(v)/len(v)) for a, v in agent_signals.items()}
fig = plot_agent_contributions(mean_signals, title='Mean Credit Signal per Agent (over 10 problems)')
plt.show()

## Step 5: Generate Preference Data for Post-Training

In [ ]:
from src.data.preference_data import generate_preference_pairs_from_traces

# Generate preference pairs from results (correct solutions = chosen)
trace_data = [{
    'problem': r['problem'],
    'final_solution': r['final_solution'],
    'correct': bool(r['correct'])
} for r in all_results]

pairs = generate_preference_pairs_from_traces(trace_data)
print(f'Generated {len(pairs)} preference pairs for DPO training')
if pairs:
    print('\nExample pair:')
    print(f'  Chosen:   {pairs[0]["chosen"][:80]}')
    print(f'  Rejected: {pairs[0]["rejected"][:80]}')

## Step 6: Evaluate Improvement (Simulated)

In real training:
1. Fine-tune each agent on its per-agent preference pairs using DPO
2. Re-run the pipeline on the same benchmark
3. Measure accuracy improvement

Here we simulate the expected improvement:

In [ ]:
import numpy as np
np.random.seed(42)
epochs = list(range(0, 6))
pre_training_acc  = tracker.summary()['accuracy']
post_training_acc = [pre_training_acc + 0.08*e + np.random.randn()*0.02 for e in epochs]

plt.figure(figsize=(8, 4))
plt.plot(epochs, post_training_acc, 'g-o', linewidth=2, markersize=8)
plt.axhline(pre_training_acc, color='red', linestyle='--', label=f'Pre-training ({pre_training_acc:.2%})')
plt.xlabel('Training Epoch'); plt.ylabel('Accuracy')
plt.title('Simulated Accuracy Improvement After AT-GRPO Post-Training')
plt.legend(); plt.grid(alpha=0.3)
plt.ylim(0, 1)
plt.show()
print(f'Pre-training accuracy:  {pre_training_acc:.2%}')
print(f'Simulated post-training: {post_training_acc[-1]:.2%}')

## Summary: Complete Pipeline

You have now implemented the full Argonne internship workflow:

1. **MAS Pipeline:** Solver → Critic → Reviser → Verifier with trace logging
2. **Benchmark Evaluation:** GSM8K accuracy, convergence rate, error correction rate
3. **Credit Assignment:** Shapley values + error localization
4. **Post-Training Data:** Preference pairs generated from agent traces
5. **Training Signals:** AT-GRPO agent/turn-level advantages
6. **Evaluation Loop:** Measure pre/post training improvement

On **Aurora**, this pipeline would run with:
- LLaMA-70B or larger as the base model
- Tensor parallelism across 8 Intel Ponte Vecchio GPUs per agent group
- 1000+ problems per training batch
- Multiple LoRA adapters per agent role

---

## Final Exercises

1. **Scale up:** Run on all 20 GSM8K problems. How does accuracy change?
2. **Debate vs Pipeline:** Compare PipelineOrchestrator vs DebateOrchestrator on the same problems
3. **LoRA adapters:** Modify to use different LoRA adapters per agent role
4. **Real data:** Try loading the actual GSM8K dataset from HuggingFace (`datasets.load_dataset('gsm8k', 'main')`)
5. **Aurora plan:** Write a 1-page plan for how you would run this on Aurora at scale